In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# --------------------------
# Interactive user input
# --------------------------
tickers_input = input("Enter stock tickers separated by commas (e.g., AAPL, TSLA, MSFT): ")
tickers = [ticker.strip().upper() for ticker in tickers_input.split(",")]

start_date_input = input("Enter start date (DD-MM-YYYY): ")
end_date_input = input("Enter end date (DD-MM-YYYY): ")

start_date = pd.to_datetime(start_date_input, format="%d-%m-%Y")
end_date = pd.to_datetime(end_date_input, format="%d-%m-%Y")

# --------------------------
# Fetch stock data (auto-adjusted)
# --------------------------
data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)
if isinstance(data.columns, pd.MultiIndex):
    data = data["Close"]

# --------------------------
# Calculate daily returns
# --------------------------
returns = data.pct_change().dropna()
avg_returns = returns.mean()
volatility = returns.std()

# --------------------------
# Portfolio weights
# --------------------------
use_equal_weights = input("\nDo you want to use equal weights for all stocks? (y/n): ").lower()
if use_equal_weights == 'y':
    weights = pd.Series(1/len(tickers), index=tickers)
    print("\nUsing equal weights:", weights.to_dict())
else:
    print("\nEnter portfolio weights for each stock (must sum to 1):")
    weights_list = []
    for ticker in tickers:
        w = float(input(f"Weight for {ticker}: "))
        weights_list.append(w)
    weights = pd.Series(weights_list, index=tickers)
    if abs(weights.sum() - 1) > 1e-6:
        print("\nWarning: Weights do not sum to 1. They will be normalized automatically.")
        weights = weights / weights.sum()

# --------------------------
# Portfolio returns
# --------------------------
portfolio_returns = (returns * weights).sum(axis=1)
portfolio_cum_returns = (1 + portfolio_returns).cumprod() - 1
portfolio_avg_return = portfolio_returns.mean()
portfolio_volatility = portfolio_returns.std()
portfolio_sharpe = portfolio_avg_return / portfolio_volatility

print("\nPortfolio Average Daily Return:", portfolio_avg_return)
print("Portfolio Daily Volatility:", portfolio_volatility)
print("Portfolio Sharpe Ratio:", portfolio_sharpe)

# --------------------------
# Generate dynamic PDF filename
# --------------------------
ticker_str = "_".join(tickers)
start_str = start_date.strftime("%d-%m-%Y")
end_str = end_date.strftime("%d-%m-%Y")
pdf_filename = f"Portfolio_Report_{ticker_str}_{start_str}_to_{end_str}.pdf"

# --------------------------
# Save plots and metrics to PDF
# --------------------------
with PdfPages(pdf_filename) as pdf:

    # Historical stock prices
    plt.figure(figsize=(12, 6))
    for ticker in tickers:
        plt.plot(data.index, data[ticker], label=ticker)
    plt.title("Historical Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Close Price (Auto-adjusted)")
    plt.legend()
    pdf.savefig()
    plt.close()

    # Cumulative returns
    cumulative_returns = (1 + returns).cumprod() - 1
    plt.figure(figsize=(12, 6))
    for ticker in tickers:
        plt.plot(cumulative_returns.index, cumulative_returns[ticker], label=ticker)
    plt.plot(portfolio_cum_returns.index, portfolio_cum_returns, label="Portfolio", linewidth=3, color="black")
    plt.title("Cumulative Returns (Stocks + Portfolio)")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Return")
    plt.legend()
    pdf.savefig()
    plt.close()

    # Correlation heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(returns.corr(), annot=True, cmap="viridis")
    plt.title("Correlation Between Stocks")
    pdf.savefig()
    plt.close()

    # Daily returns comparison
    plt.figure(figsize=(12, 6))
    for ticker in tickers:
        plt.plot(returns.index, returns[ticker], alpha=0.6, label=ticker)
    plt.plot(portfolio_returns.index, portfolio_returns, color='black', linewidth=2.5, label='Portfolio')
    plt.title("Daily Returns: Stocks vs Portfolio")
    plt.xlabel("Date")
    plt.ylabel("Daily Return")
    plt.legend()
    pdf.savefig()
    plt.close()

    # Metrics page
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis('off')
    metrics_text = f"""
    Portfolio Report
    ---------------------------
    Tickers: {tickers}
    Portfolio Weights: {weights.to_dict()}
    
    Portfolio Average Daily Return: {portfolio_avg_return:.6f}
    Portfolio Daily Volatility: {portfolio_volatility:.6f}
    Portfolio Sharpe Ratio: {portfolio_sharpe:.2f}
    """
    ax.text(0.05, 0.5, metrics_text, fontsize=12, va='center')
    pdf.savefig()
    plt.close()

print(f"\nPDF report saved as: {pdf_filename}")
